In [1]:
#Python Imports
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
import re
from pandas.io.json import json_normalize
import json
import pandas as pd
import numpy as np

from surprise import SVD, accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import NMF

from pandas.io.json import json_normalize
from pymongo import MongoClient
from collections import defaultdict
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import re
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval

In [11]:
def convert_ids(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('int64')

def convert_to_float(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('float64')

def to_json(csv_entry):
    return json.loads(re.sub('\'', '"', csv_entry))


def get_top_n(predictions, n=200):
    '''SUPRISE API
    Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_movie_name(movie_id):
    return ratings_with_movie_names[ratings_with_movie_names.id == movie_id]['title'].iloc[0]


def print_user_prediction(userId, predictions_dict, meta_df):
    users_viewed_movies = ratings_with_movie_names[ratings_with_movie_names['userId'] == userId][
        ['rating', 'original_title']]
    #print(f'User {userId} has viewed the following movies:\n')

    for row in users_viewed_movies.itertuples():
        rating = row[1]
        original_title = row[2]
        #print(f'\t{original_title}, Rating: {rating}')

    #print(f'\nThe following movies are recommended for User {userId}\n')
    recommended_movies = [get_movie_name(mov_id[0], meta_df) for mov_id in predictions_dict[userId]]

    #for movie in recommended_movies:
        #print(f'\t{movie}')

def get_movie_name(movie_id, movie_meta_df):
    return movie_meta_df[movie_meta_df.id == movie_id]['title'].iloc[0]

def get_movie_id(title, movie_meta_df):
    return movie_meta_df[movie_meta_df.title == title]['id'].iloc[0]


def get_all_movies_in_cluster(cluster_number, cluster_dict, meta_df):
    movies = cluster_dict[cluster_number]
    return [get_movie_name(mov, meta_df) for mov in movies]

####

# cluster_distribution = [len(movies) for (clust, movies) in movie_summaries_clustered.items()]

def get_cluster_number(movie, cluster_zip):
    for cluster, movie_id in cluster_zip:

        if movie_id == movie:
            return cluster

    raise Exception('Movie not found in cluster')

In [12]:
movies_metadata_df1 = pd.read_csv('the-movies-dataset/movies_metadata.csv'
                                 , converters={ 'id': lambda x: convert_ids(x)
                                               , 'imdb_id': lambda x: convert_ids(x)
                                               ,'popularity': lambda x: convert_to_float(x)
                                               ,'genres': lambda x: to_json(x)}
                                 , usecols=['id', 'original_title'
                                                , 'genres' #'homepage'
                                                , 'overview', 'popularity', 'poster_path'
                                                , 'release_date', 'revenue', 'runtime'
                                                , 'spoken_languages', 'title'
                                                , 'vote_average', 'vote_count']
                                , dtype={'populariy': np.float64}
                                , parse_dates=True)


movies_lookup_df = pd.read_csv('the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'title'])

#####################################
##SVD DATA SET
movies_df = pd.read_csv('the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'original_title', 'belongs_to_collection'
                                 , 'budget', 'genres', 'homepage'
                                 ,'imdb_id', 'overview', 'popularity', 'poster_path'
                                 , 'production_companies','release_date', 'revenue', 'runtime',
                                 'spoken_languages', 'status', 'tagline', 'title', 'video',
                                 'vote_average', 'vote_count'])
#####################################

ratings_df = pd.read_csv('the-movies-dataset/ratings_small.csv')



content_filter_df = pd.read_pickle('content_filter_df.pkl')
content_filter_df = content_filter_df[['id',
 'popularity',
 #'release_date',
 'vote_average',
 'release_year',
 0,1,2,3,4,5, 6, 7, 8,9,10,11,12,13,14,15,16,17,18,19]]
content_filter_df = content_filter_df.dropna()

idx = pd.Index(content_filter_df['id'])
idx
content_filter_df.index = idx

/home/chaitya/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
###May need Fuzzy matching, but for now:
movies_df = movies_df[movies_df.spoken_languages == """[{'iso_639_1': 'en', 'name': 'English'}]"""]

In [14]:
ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')
ratings_with_movie_names = ratings_with_movie_names[ratings_with_movie_names.original_title.isnull() == False]

In [15]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_with_movie_names[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

# SVD : Collaborative Filtering

In [16]:
algo = SVD(verbose=True)
algo.fit(trainset)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=False)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


{'fit_time': (1.1008143424987793,
  1.3403854370117188,
  1.3835294246673584,
  1.3081231117248535,
  0.9806644916534424),
 'test_mae': array([0.68827432, 0.696937  , 0.69575826, 0.6892278 , 0.68536742]),
 'test_rmse': array([0.89184134, 0.90071568, 0.90493053, 0.89220854, 0.88670869]),
 'test_time': (0.035508155822753906,
  0.034493446350097656,
  0.03553271293640137,
  0.03453516960144043,
  0.030032634735107422)}

In [17]:
predictions = algo.test(testset)
### Tune this value to get fewer results faster, but less options to choose from
top_n = get_top_n(predictions)


predicted_movies_by_name = defaultdict(list)

### This builds the dictionary of predicted movies for all users
for key, value in top_n.items():
    predicted_movies_by_name[key] = [get_movie_name(mov_id[0], movies_metadata_df1) for mov_id in value]

In [9]:
from collections import namedtuple
UserFavoriteRating = namedtuple('UserFavoriteRating', ['title', 'rating'])

def users_top_n_movies(n, userId, predictions_dict, meta_df):
    users_viewed_movies = ratings_with_movie_names[ratings_with_movie_names['userId'] == userId][['rating', 'original_title']]
    
    viewed_movies = []

    for row in users_viewed_movies.itertuples():
        rating = row[1]
        original_title = row[2]
        film = UserFavoriteRating(original_title, rating)
        viewed_movies.append(film)
    
    sorted(viewed_movies, key=lambda film: film[1])
    
    return viewed_movies[0:n]

# GET A USERS TOP RATED MOVIES

In [10]:
users_top_n_movies(5, 321, predicted_movies_by_name, movies_metadata_df1)

[UserFavoriteRating(title='The Endless Summer', rating=5.0),
 UserFavoriteRating(title='Rumble Fish', rating=5.0),
 UserFavoriteRating(title='A River Runs Through It', rating=4.0),
 UserFavoriteRating(title='Terminator 3: Rise of the Machines', rating=5.0),
 UserFavoriteRating(title='The Poseidon Adventure', rating=5.0)]

# At this point, you should have user personas to get a pool of movies to choose from, not simply pre-made users:

In [11]:
print_user_prediction(47, top_n, movies_metadata_df1)

User 47 has viewed the following movies:

	48 Hrs., Rating: 4.0
	Back to the Future Part II, Rating: 4.0
	20,000 Leagues Under the Sea, Rating: 5.0
	Muriel's Wedding, Rating: 4.0
	High Noon, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 4.0
	Faster, Pussycat! Kill! Kill!, Rating: 4.0
	Silent Hill, Rating: 4.0
	The Conversation, Rating: 3.0
	To Kill a Mockingbird, Rating: 3.0

The following movies are recommended for User 47

	Sleepless in Seattle
	Murder She Said
	The Million Dollar Hotel
	Galaxy Quest
	The Thomas Crown Affair
	Nell
	Confession of a Child of the Century
	Men in Black II
	Birdman of Alcatraz
	While You Were Sleeping
	Shriek If You Know What I Did Last Friday the Thirteenth
	Edward Scissorhands
	The Thomas Crown Affair
	Space Jam
	Boogie Nights
	The Return of the King
	Ghost Rider
	Fools Rush In
	Totally Blonde
	Rumble Fish
	Flags of Our Fathers
	Dawn of the Dead
	The Bank Dick
	Bonnie and Clyde
	License to Wed
	A River Runs Through It
	Hard Target
	Straw Dogs

In [12]:
UserFavoriteRating = namedtuple('UserFavoriteRating', ['title', 'rating'])
def collab_filter_recommendations(user, top_ns, movie_meta_df):
    
    predictions = top_ns[user]
    
    return [UserFavoriteRating(get_movie_name(pred[0], movie_meta_df), pred[1]) for pred in predictions]

In [13]:
collab_filter_recommendations(47, top_n, movies_metadata_df1)

[UserFavoriteRating(title='Sleepless in Seattle', rating=5),
 UserFavoriteRating(title='Murder She Said', rating=4.799372537101847),
 UserFavoriteRating(title='The Million Dollar Hotel', rating=4.714216916801487),
 UserFavoriteRating(title='Galaxy Quest', rating=4.668626119527115),
 UserFavoriteRating(title='The Thomas Crown Affair', rating=4.667413193529942),
 UserFavoriteRating(title='Nell', rating=4.581602817724114),
 UserFavoriteRating(title='Confession of a Child of the Century', rating=4.578217597412764),
 UserFavoriteRating(title='Men in Black II', rating=4.569532225783706),
 UserFavoriteRating(title='Birdman of Alcatraz', rating=4.55974725062475),
 UserFavoriteRating(title='While You Were Sleeping', rating=4.549629449802998),
 UserFavoriteRating(title='Shriek If You Know What I Did Last Friday the Thirteenth', rating=4.539035153181185),
 UserFavoriteRating(title='Edward Scissorhands', rating=4.532755623125054),
 UserFavoriteRating(title='The Thomas Crown Affair', rating=4.52607

# CONTENT FILTERING

In [1]:
movies_metadata_df = pd.read_csv('the-movies-dataset/movies_metadata.csv'
                                 , converters={ 'id': lambda x: convert_ids(x)
                                               , 'imdb_id': lambda x: convert_ids(x)
                                               ,'popularity': lambda x: convert_to_float(x)
                                               ,'genres': lambda x: to_json(x)}
                                 , usecols=['id', 'original_title'
                                                , 'genres' #'homepage'
                                                , 'overview', 'popularity', 'poster_path'
                                                , 'release_date', 'revenue', 'runtime'
                                                , 'spoken_languages', 'title'
                                                , 'vote_average', 'vote_count']
                                , dtype={'populariy': np.float64}
                                , parse_dates=True)
movies_metadata_df = movies_metadata_df.drop_duplicates(subset=['id'])

NameError: name 'pd' is not defined

In [15]:
def get_recommendations(title, tfidf_matrix):
    # Get the index of the movie that matches the title
    idx = movies_df[title]
    cosine_sim = linear_kernel(tfidf_matrix[idx], tfidf_matrix)
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[0]))
    
    pickle.dump(tfidf_matrix ,open("feature.pkl","wb"))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[0:21]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    #print(idx, title , movies_metadata_df['title'].iloc[idx],movies_metadata_df['genres'].iloc[idx])
    return movies_metadata_df['title'].iloc[movie_indices]

In [16]:
movies_metadata_df.shape

(45434, 13)

In [17]:
# Load keywords and credits
credits = pd.read_csv('the-movies-dataset/credits.csv')
keywords = pd.read_csv('the-movies-dataset/keywords.csv')

credits = credits.drop_duplicates(subset=['id'])
keywords = keywords.drop_duplicates(subset=['id'])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies_metadata_df['id'] = movies_metadata_df['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
movies_metadata_df = credits.merge(movies_metadata_df, on='id')
movies_metadata_df = movies_metadata_df.merge(keywords, on='id')


In [18]:
movies_df = pd.Series(movies_metadata_df.index, index=movies_metadata_df['title']).drop_duplicates()

In [19]:
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    movies_metadata_df[feature] = movies_metadata_df[feature].apply(literal_eval)

ValueError: malformed node or string: [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

In [20]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [21]:
# Define new director, cast, genres and keywords features that are in a suitable form.
movies_metadata_df['director'] = movies_metadata_df['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    movies_metadata_df[feature] = movies_metadata_df[feature].apply(get_list)

In [22]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    movies_metadata_df[feature] = movies_metadata_df[feature].apply(clean_data)

# Create a new soup feature
movies_metadata_df['soup'] = movies_metadata_df.apply(create_soup, axis=1)

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movies_metadata_df['soup'])
#count_matrix
cosine_sim = linear_kernel(count_matrix[0], count_matrix)
cosine_sim

array([[10.,  1.,  1., ...,  0.,  0.,  0.]])

In [23]:
print(get_recommendations('Toy Story',count_matrix))
print_user_prediction(47, top_n, movies_metadata_df)

0                                         Toy Story
2996                                    Toy Story 2
15342                                   Toy Story 3
27373                                         Anina
21914                          Toy Story of Terror!
25780                               Partysaurus Rex
25782                    Toy Story That Time Forgot
3308                              Creature Comforts
11111                                 Monster House
11578                     The Ugly Duckling and Me!
13505    Kung Fu Panda: Secrets of the Furious Five
17353                                        Cars 2
24903                            Jetsons: The Movie
25446                   Scooby-Doo! Abracadabra-Doo
34406                 Jis Desh Mein Ganga Rehta Hai
350                                 The Flintstones
369                                     Ri¢hie Ri¢h
405                            Addams Family Values
581                                         Aladdin
608         

# Graph Filter

In [24]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install neomodel

Note: you may need to restart the kernel to use updated packages.


In [27]:
from neo4j import GraphDatabase

uri = "bolt://localhost:11002"
driver = GraphDatabase.driver(uri, auth=('neo4j', 'ayush'))

In [31]:
session = driver.session()
query = "MATCH (n) RETURN count(n)"
no_count = session.run(query)
for i in no_count:
    print(i)
print(no_count)

<Record count(n)=10371>


In [32]:
import json
from datetime import datetime
from pprint import pprint
import codecs
import os
import json
import logging
import random
from py2neo import Graph, Path, Node, Relationship, cypher

In [39]:
#walk_limit defines the number of nodes it can hop
walk_limit = 5 

#All types of relationships possible, will be helpful in defining probabilities
relationships = ["FAVORITE","GENRES","SIMILAR","WATCHED"]

In [40]:
def generateRandomPath():
    #query to select a random node to start with
    query = '''
        START t=node(*) 
        MATCH (a)-[]->(t) 
        RETURN a
        SKIP {n} LIMIT 1'''
    
    #get total number of nodes
    node_count_query =  "MATCH (n) RETURN count(n)"
    node_count = 10371
    session = driver.session()
    
    #select a random node
    #random_node = (session.run(query,{"n":random.randint(0,node_count - 1), "m":1}))
    random_node = (session.run(query,{"n":862, "m":1}))

    #print(type(random_node))
    #x = random_node.single()[0]
#     print(x)
#     print(x.id)
    #print(random_node.single()[0])
    
#     for i in random_node:
#         print (i)
    r_node =  random_node.single()[0]
    return walk(r_node, 0) # start random walk and return the final node

In [41]:
path = []

In [42]:
def walk(start_node, current_length, prev_node=None, max_length = walk_limit):
    #print(type(start_node.labels))
    if (start_node.labels == {'Movies'}):
        #print(start_node)
        path.append(start_node)
    else:
        current_length-=1
    #return if path length exceed limit
    if (current_length >= max_length):
        return {start_node:current_length}
    
    #query to get all incoming and outgoing paths from a node
    query = """match p=(n)-[]-() where id(n)={id} return p"""
    session = driver.session()
    rels = session.run(query,{"id":start_node.id})
#     for i in rels:
#         print(i)
    #Convert the output to a list
    relationships = rels.data()
    #print(relationships)
    size = len(relationships)
    
    #select a random node
    node = random.randint(0,size-1)
    next_node = relationships[node]["p"].end_node
    #print(relationships[node]["p"].end_node)
    current_node = relationships[node]["p"].start_node
    
    #If next random node is similar to previous node, choose another
    if(next_node.id == current_node.id and size > 1):
        node = random.randint(0,size-1)
        next_node = relationships[node]["p"].end_node
        current_node = relationships[node]["p"].start_node
    #if previous node is the only possible path, terminate
    elif(next_node.id == current_node.id and size == 1):
        
        return {current_node:current_length}
        
    return walk(next_node, current_length + 1, current_node)

In [43]:
print(generateRandomPath())
#for i in range(5):
    
for i in path:
    print(i)

{<Node id=728 labels={'Movies'} properties={'rating_mean': '3.5833333333333335', 'title': 'Flirting With Disaster (1996)', 'movieId': '125'}>: 5}
<Node id=3385 labels={'Movies'} properties={'rating_mean': '4.0375', 'title': 'Road Warrior, The (Mad Max 2) (1981)', 'movieId': '3703'}>
<Node id=2456 labels={'Movies'} properties={'rating_mean': '3.383720930232558', 'title': 'Fly, The (1986)', 'movieId': '2455'}>
<Node id=1803 labels={'Movies'} properties={'rating_mean': '2.94', 'title': 'Event Horizon (1997)', 'movieId': '1590'}>
<Node id=1815 labels={'Movies'} properties={'rating_mean': '2.4615384615384617', 'title': 'Mimic (1997)', 'movieId': '1603'}>
<Node id=3480 labels={'Movies'} properties={'rating_mean': '2.2948717948717947', 'title': 'Hollow Man (2000)', 'movieId': '3826'}>
<Node id=728 labels={'Movies'} properties={'rating_mean': '3.5833333333333335', 'title': 'Flirting With Disaster (1996)', 'movieId': '125'}>


# GRAPH CODE HERE

In [28]:
def get_count_first_degree_films_of(tx, title):
    for record in tx.run("MATCH (origin:MOVIE)-[:APPEARED_IN]-(actor)-[:APPEARED_IN]-(first_movie:MOVIE)"
                         "WHERE origin.title = {title} "
                         "RETURN count(*)", title=title):
        print(record[0])
        
def get_first_degree_films_of(tx, title):
    nodes = []
    
    for record in tx.run("MATCH (origin:MOVIE)-[:APPEARED_IN]-(actor)-[:APPEARED_IN]-(first_movie:MOVIE)"
                         "WHERE origin.title = {title} "
                         "RETURN first_movie", title=title):
        
        nodes.append(record.data())
    
    return nodes

first_degree_away = None
with driver.session() as session:
    first_degree_away = session.read_transaction(get_first_degree_films_of, 'Superman')

IndexError: list index out of range

In [16]:
GraphMember = namedtuple('GraphMember', ['title', 'movie_id'])

def neo4j_results_to_tuples(results):
    return [GraphMember(node['first_movie'].get('title'), node['first_movie'].get('movie_id')) for node in results]

NameError: name 'namedtuple' is not defined

In [ ]:
neo4j_results_to_tuples(first_degree_away)

In [ ]:
def get_first_degree_films_of(tx, title):
    nodes = []

    for record in tx.run("MATCH (origin:MOVIE)-[:APPEARED_IN]-(actor)-[:APPEARED_IN]-(first_movie:MOVIE)"
                         "WHERE origin.title = {title} "
                         "RETURN first_movie", title=title):
        nodes.append(record.data())

    return nodes

# Get First Degree Away Films of Sleepless in Seattle

In [ ]:
first_degree_away = None
with driver.session() as session:
    first_degree_away = session.read_transaction(get_first_degree_films_of, 'Sleepless in Seattle')

In [ ]:
def neo4j_results_to_tuples(results):
    return [GraphMember(node['first_movie'].get('title'), node['first_movie'].get('movie_id')) for node in results]

In [ ]:
def get_connected_movies(list_favorite_movies, driver):
    list_connected_movies = []

    with driver.session() as session:
        for movie in list_favorite_movies:
            first_degree_away_films = session.read_transaction(get_first_degree_films_of, movie)
            film_tups = neo4j_results_to_tuples(first_degree_away_films)
            list_connected_movies.extend(film_tups)

    return list_connected_movies

## Run a test with a user

In [ ]:
user_number = 321
predictions_dict = predicted_movies_by_name
get_graph_on = users_top_n_movies(10, user_number, predictions_dict, movies_metadata_df)

In [ ]:
get_graph_on

In [ ]:
def get_top_three_favs(user_id):
    
    favorite_seen_movie_array = users_top_n_movies(200, user_id, predicted_movies_by_name, movies_metadata_df1)
    sorted_seen_movies = sorted(favorite_seen_movie_array, key=lambda k: k[1], reverse=True)
    return sorted_seen_movies[0:3]

In [ ]:
get_top_three_favs(47)

# Collab Filter List

In [ ]:
collab_filtered_movies = sorted(collab_filter_recommendations(47, top_n, movies_metadata_df1), key=lambda k: k[1], reverse=True)
collab_filtered_movies

## Content Filtered

In [ ]:
# #get_movie_id('The Endless Summer', movies_metadata_df1)
# content_filtered_tups = top_n_closest_content_filtered(50, 321, content_filter_df)
# content_filtered_tups

In [ ]:
content_filtered_tups = get_recommendations('Sherlock Holmes',count_matrix)
content_filtered_tups

In [ ]:
favorite_list = get_top_three_favs(321)
movie_names = [movie[0] for movie in favorite_list]

In [ ]:
first_degree_away_films = get_connected_movies([mov[0] for mov in top_three], driver)

In [ ]:
graph_id_content_filter = set({movie[1] for movie in first_degree_away})

In [ ]:
graph_name_collab_filter = set({movie[0] for movie in first_degree_away})

In [ ]:
content_id_set = set(movie[0] for movie in content_filtered_tups)

In [ ]:
collab_movie_name_set = set(movie[0] for movie in collab_filtered_movies)

In [ ]:
graph_name_collab_filter & collab_movie_name_set

In [ ]:
content_id_set & graph_id_content_filter

In [ ]:
get_movie_name(550, movies_metadata_df1)